<h1> ECE4179 - Semi-Supervised Learning Project</h1>
<h2>Data</h2>

We will be using a dataset that can be obtained directly from the torchvision package. There are 10 classes and we will be training a CNN for the image classification task. We have training, validation and test sets that are labelled with the class, and a large unlabeled set.

We will simulating a low training data scenario by only sampling a small percentage of the labelled data (10%) as training data. The remaining examples will be used as the validation set.

To get the labelled data, change the dataset_dir to something suitable for your machine, and execute the following (you will then probably want to wrap the dataset objects in a PyTorch DataLoader):

In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import STL10 as STL10
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torchvision
from torch.utils.data import DataLoader
from copy import deepcopy
from torch.optim import Adam
import torch.optim as optim
from torchvision import models
from sklearn.metrics import f1_score, classification_report

####### CHANGE TO APPROPRIATE DIRECTORY TO STORE DATASET
dataset_dir = "../../CNN-VAE/data"
#For MonARCH
# dataset_dir = "/mnt/lustre/projects/ds19/SHARED"

#All images are 3x96x96
image_size = 96
#Example batch size
batch_size = 32

<h3>Create the appropriate transforms</h3>

In [2]:
#Perform random crops and mirroring for data augmentation
transform_train = transforms.Compose(
    [transforms.RandomCrop(image_size, padding=4),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_unlabelled = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#No random 
transform_test = transforms.Compose(
    [transforms.CenterCrop(image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


<h3>Create training and validation split</h3>

In [3]:
#Load train and validation sets
trainval_set = STL10(dataset_dir, split='train', transform=transform_train, download=True)

#Use 10% of data for training - simulating low data scenario
num_train = int(len(trainval_set)*0.1)

#Split data into train/val sets
torch.manual_seed(0) #Set torch's random seed so that random split of data is reproducible
train_set, val_set = random_split(trainval_set, [num_train, len(trainval_set)-num_train])

#Load test set
test_set = STL10(dataset_dir, split='test', transform=transform_test, download=False)

Files already downloaded and verified


<h3>Get the unlabelled data</h3>

In [4]:
unlabelled_set = STL10(dataset_dir, split='unlabeled', transform=transform_unlabelled, download=True)

Files already downloaded and verified


You may find later that you want to make changes to how the unlabelled data is loaded. This might require you sub-classing the STL10 class used above or to create your own dataloader similar to the Pytorch one.
https://pytorch.org/docs/stable/_modules/torchvision/datasets/stl10.html#STL10

<h3>Create the four dataloaders</h3>

In [5]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
unlabelled_loader = DataLoader(unlabelled_set, shuffle=True, batch_size=batch_size)

valid_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

<h3>Accuracy</h3>

In [6]:
# Define the test function
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")

<h3>Marco F1 Score</h3>

In [7]:
# Define the test function to calculate F1 score
def test_model_with_f1(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            # Collect all predictions and labels for F1-score calculation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate the Macro F1-score for each class
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    # Alternatively, you can get a detailed report for all classes
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in range(10)])
    
    print(f"Macro F1-score: {f1}")
    print("Classification Report:\n", report)

## Network

Let's use a ResNet18 architecture for our CNN...

## ResNet18

In [8]:
# We will keep this for later
model0 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


for name, param in model0.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

Using cache found in C:\Users\weita/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\weita\anaconda3\envs\env_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\weita\anaconda3\envs\env_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Name: conv1.weight, Shape: torch.Size([64, 3, 7, 7])
Name: bn1.weight, Shape: torch.Size([64])
Name: bn1.bias, Shape: torch.Size([64])
Name: layer1.0.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.0.bn1.weight, Shape: torch.Size([64])
Name: layer1.0.bn1.bias, Shape: torch.Size([64])
Name: layer1.0.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.0.bn2.weight, Shape: torch.Size([64])
Name: layer1.0.bn2.bias, Shape: torch.Size([64])
Name: layer1.1.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.1.bn1.weight, Shape: torch.Size([64])
Name: layer1.1.bn1.bias, Shape: torch.Size([64])
Name: layer1.1.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.1.bn2.weight, Shape: torch.Size([64])
Name: layer1.1.bn2.bias, Shape: torch.Size([64])
Name: layer2.0.conv1.weight, Shape: torch.Size([128, 64, 3, 3])
Name: layer2.0.bn1.weight, Shape: torch.Size([128])
Name: layer2.0.bn1.bias, Shape: torch.Size([128])
Name: layer2.0.conv2.weight, Shape: torch.Size(

In [9]:
# add your code below. Define the number of epochs to train for based on your laptop's performance
# on a CPU, my machine takes about 2 minutes per epoch
num_epochs=10

# Create a new model from the pretrained one
model_resnet18 = deepcopy(model0)  # <--- study the code here

# Modify the last fully connected layer to match the number of classes (e.g., 10 for STL10)
num_classes = 10
model_resnet18.fc = nn.Linear(model_resnet18.fc.in_features, num_classes)

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model_resnet18.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_resnet18.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model_resnet18(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Evaluation on test set
model_resnet18.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_resnet18(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print test accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")

Epoch [1/10], Loss: 1.5088257975876331
Epoch [2/10], Loss: 1.0582835879176855
Epoch [3/10], Loss: 0.6645623072981834
Epoch [4/10], Loss: 0.4580966904759407
Epoch [5/10], Loss: 0.320215062238276
Epoch [6/10], Loss: 0.33488638303242624
Epoch [7/10], Loss: 0.35667672008275986
Epoch [8/10], Loss: 0.31174362311139703
Epoch [9/10], Loss: 0.28476891526952386
Epoch [10/10], Loss: 0.339187182020396
Test Accuracy: 60.5125%


In [10]:
# Call the test function
test_model(model_resnet18, test_loader)

Test Accuracy: 60.5125%


In [11]:
# Call the function to calculate and print F1-scores
test_model_with_f1(model_resnet18, test_loader)

Macro F1-score: 0.6034147584709965
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.80      0.66      0.72       800
     Class 1       0.79      0.52      0.63       800
     Class 2       0.82      0.59      0.68       800
     Class 3       0.48      0.54      0.51       800
     Class 4       0.51      0.76      0.61       800
     Class 5       0.44      0.51      0.47       800
     Class 6       0.83      0.34      0.49       800
     Class 7       0.59      0.54      0.56       800
     Class 8       0.68      0.72      0.70       800
     Class 9       0.53      0.87      0.66       800

    accuracy                           0.61      8000
   macro avg       0.65      0.61      0.60      8000
weighted avg       0.65      0.61      0.60      8000



## EfficientNet

In [12]:
# Load pretrained EfficientNet-B0 model from torchvision hub
efficientnetb0 = torch.hub.load('pytorch/vision', 'efficientnet_b0', weights="EfficientNet_B0_Weights.IMAGENET1K_V1")

for name, param in efficientnetb0.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

Name: features.0.0.weight, Shape: torch.Size([32, 3, 3, 3])
Name: features.0.1.weight, Shape: torch.Size([32])
Name: features.0.1.bias, Shape: torch.Size([32])
Name: features.1.0.block.0.0.weight, Shape: torch.Size([32, 1, 3, 3])
Name: features.1.0.block.0.1.weight, Shape: torch.Size([32])
Name: features.1.0.block.0.1.bias, Shape: torch.Size([32])
Name: features.1.0.block.1.fc1.weight, Shape: torch.Size([8, 32, 1, 1])
Name: features.1.0.block.1.fc1.bias, Shape: torch.Size([8])
Name: features.1.0.block.1.fc2.weight, Shape: torch.Size([32, 8, 1, 1])
Name: features.1.0.block.1.fc2.bias, Shape: torch.Size([32])
Name: features.1.0.block.2.0.weight, Shape: torch.Size([16, 32, 1, 1])
Name: features.1.0.block.2.1.weight, Shape: torch.Size([16])
Name: features.1.0.block.2.1.bias, Shape: torch.Size([16])
Name: features.2.0.block.0.0.weight, Shape: torch.Size([96, 16, 1, 1])
Name: features.2.0.block.0.1.weight, Shape: torch.Size([96])
Name: features.2.0.block.0.1.bias, Shape: torch.Size([96])
Nam

Using cache found in C:\Users\weita/.cache\torch\hub\pytorch_vision_main


In [13]:
# Modify the last fully connected layer to output 10 classes
num_classes = 10
efficientnetb0.classifier[1] = nn.Linear(efficientnetb0.classifier[1].in_features, num_classes)

# Move the model to the appropriate device (GPU/CPU)
efficientnetb0 = efficientnetb0.to(device)

# Define loss function (CrossEntropyLoss) and optimizer (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnetb0.parameters(), lr=0.001)

# Training and validation code
def train_model(model, train_loader, valid_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Loop through batches in the training data
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Calculate running loss
            running_loss += loss.item()

        # Validation after every epoch
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {accuracy}%")

train_model(efficientnetb0, train_loader, valid_loader, num_epochs=10)

Epoch 1/10, Loss: 1.6454911679029465, Accuracy: 69.8%
Epoch 2/10, Loss: 0.5324282217770815, Accuracy: 72.2%
Epoch 3/10, Loss: 0.26369882468134165, Accuracy: 72.64444444444445%
Epoch 4/10, Loss: 0.24109109956771135, Accuracy: 71.77777777777777%
Epoch 5/10, Loss: 0.26976340916007757, Accuracy: 71.04444444444445%
Epoch 6/10, Loss: 0.32619810523465276, Accuracy: 70.17777777777778%
Epoch 7/10, Loss: 0.2994155599735677, Accuracy: 68.62222222222222%
Epoch 8/10, Loss: 0.3010925780981779, Accuracy: 71.02222222222223%
Epoch 9/10, Loss: 0.17950862017460167, Accuracy: 71.5111111111111%
Epoch 10/10, Loss: 0.146677186479792, Accuracy: 71.57777777777778%


In [14]:
# Call the test function
test_model(efficientnetb0, test_loader)

Test Accuracy: 73.0625%


In [15]:
# Call the function to calculate and print F1-scores
test_model_with_f1(efficientnetb0, test_loader)

Macro F1-score: 0.7225164368992557
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.82      0.84      0.83       800
     Class 1       0.79      0.76      0.77       800
     Class 2       0.72      0.96      0.82       800
     Class 3       0.64      0.67      0.66       800
     Class 4       0.66      0.81      0.73       800
     Class 5       0.67      0.39      0.49       800
     Class 6       0.68      0.81      0.74       800
     Class 7       0.66      0.67      0.67       800
     Class 8       0.86      0.87      0.86       800
     Class 9       0.86      0.53      0.66       800

    accuracy                           0.73      8000
   macro avg       0.74      0.73      0.72      8000
weighted avg       0.74      0.73      0.72      8000



## Vision Transformer (ViT)

In [16]:
# Set image size to 224x224 to match the input size of ViT
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_unlabelled = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load train and validation sets without redownloading data
trainval_set = STL10(dataset_dir, split='train', transform=transform_train, download=False)

# Use 10% of the data for training (simulating a low data scenario)
num_train = int(len(trainval_set) * 0.1)

# Split data into train/validation sets with a fixed random seed
torch.manual_seed(0)  # Ensure reproducibility
train_set, val_set = random_split(trainval_set, [num_train, len(trainval_set) - num_train])

# Load test set without redownloading data
test_set = STL10(dataset_dir, split='test', transform=transform_test, download=False)

# Create DataLoader for train, validation, and test sets
train_loader = DataLoader(train_set, shuffle=True, batch_size=32)
valid_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [17]:
# Load pretrained Vision Transformer (ViT) model from torchvision models
vit = models.vit_b_16(pretrained=True)

# Print the model structure to verify the changes
for name, param in vit.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

C:\Users\weita\anaconda3\envs\env_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\weita\anaconda3\envs\env_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Name: class_token, Shape: torch.Size([1, 1, 768])
Name: conv_proj.weight, Shape: torch.Size([768, 3, 16, 16])
Name: conv_proj.bias, Shape: torch.Size([768])
Name: encoder.pos_embedding, Shape: torch.Size([1, 197, 768])
Name: encoder.layers.encoder_layer_0.ln_1.weight, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_1.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.self_attention.in_proj_weight, Shape: torch.Size([2304, 768])
Name: encoder.layers.encoder_layer_0.self_attention.in_proj_bias, Shape: torch.Size([2304])
Name: encoder.layers.encoder_layer_0.self_attention.out_proj.weight, Shape: torch.Size([768, 768])
Name: encoder.layers.encoder_layer_0.self_attention.out_proj.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_2.weight, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_2.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.mlp.0.weight, Shape: torch.Size([3072, 768])
Name: encoder.layers.

In [18]:
# Modify the last fully connected layer to match the number of classes (e.g., 10 classes)
num_classes = 10
vit.heads.head = nn.Linear(vit.heads.head.in_features, num_classes)

# Freeze all layers except the last fully connected layer (heads.head)
for name, param in vit.named_parameters():
    if 'heads.head' in name:  # Only unfreeze the heads.head layer
        param.requires_grad = True
        print(f"Unfreezing layer: {name}")
    else:
        param.requires_grad = False

# Move the model to the appropriate device (GPU/CPU)
vit = vit.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropy for classification
optimizer = optim.Adam(vit.parameters(), lr=0.001)

# Training and validation code
def train_model(model, train_loader, valid_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Loop through batches in the training data
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Calculate running loss
            running_loss += loss.item()

        # Validation after every epoch
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {accuracy}%")

# Use your train_loader and valid_loader
train_model(vit, train_loader, valid_loader, num_epochs=10)

Unfreezing layer: heads.head.weight
Unfreezing layer: heads.head.bias


C:\Users\weita\anaconda3\envs\env_gpu\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 1/10, Loss: 1.67296726629138, Accuracy: 89.37777777777778%
Epoch 2/10, Loss: 0.674645284190774, Accuracy: 93.57777777777778%
Epoch 3/10, Loss: 0.34950040467083454, Accuracy: 95.88888888888889%
Epoch 4/10, Loss: 0.2200820599682629, Accuracy: 96.33333333333333%
Epoch 5/10, Loss: 0.1608294164761901, Accuracy: 96.57777777777778%
Epoch 6/10, Loss: 0.12484467728063464, Accuracy: 96.88888888888889%
Epoch 7/10, Loss: 0.10040531447157264, Accuracy: 97.06666666666666%
Epoch 8/10, Loss: 0.08038817113265395, Accuracy: 96.86666666666666%
Epoch 9/10, Loss: 0.06875781808048487, Accuracy: 97.28888888888889%
Epoch 10/10, Loss: 0.058618630515411496, Accuracy: 96.84444444444445%


In [19]:
# Call the test function
test_model(vit, test_loader)

Test Accuracy: 97.3625%


In [20]:
# Call the function to calculate and print F1-scores
test_model_with_f1(vit, test_loader)

Macro F1-score: 0.9736377311179627
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.98      0.98      0.98       800
     Class 1       0.99      0.97      0.98       800
     Class 2       0.99      0.98      0.99       800
     Class 3       0.97      0.97      0.97       800
     Class 4       0.98      0.93      0.95       800
     Class 5       0.94      0.97      0.95       800
     Class 6       0.94      0.97      0.95       800
     Class 7       0.98      0.99      0.99       800
     Class 8       0.98      0.98      0.98       800
     Class 9       0.98      0.99      0.99       800

    accuracy                           0.97      8000
   macro avg       0.97      0.97      0.97      8000
weighted avg       0.97      0.97      0.97      8000

